In [ ]:
from __future__ import print_function
from ortools.sat.python import cp_model

class RoomsPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_rooms, num_lectures, num_shifts, sols):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_rooms = num_rooms
        self._num_lectures = num_lectures
        self._num_shifts = num_shifts
        self._solutions = set(sols)
        self._solution_count = 0
        
    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            print('Solution %i' % self._solution_count)
            for s in range(self._num_shifts):
                print('Shift %i' % s)
                for l in range(self._num_lectures):
                    is_allocated = False
                    for r in range(self._num_rooms): 
                        if self.Value(self._shifts[(l, s, r)]):
                            is_allocated = True
                            print('  Lecture %i occupies room %i' % (l, r))
                    if not is_allocated:
                        print('  Lecture %i not allocated ' % l)
            print()
        self._solution_count += 1  
        
    def solution_count(self):
            return self._solution_count
        

def main():
    # Data.
    num_rooms = 5
    num_lectures = 6
    num_shifts = 3
    all_rooms = range(num_rooms)
    all_shifts = range(num_shifts)
    all_lectures = range(num_lectures)
    
    # Create the model
    model = cp_model.CpModel()
    
    # Creating shift variables
    shifts = {}
    for l in all_lectures:
        for s in all_shifts:
            for r in all_rooms:
                shifts[(l, s, r)] = model.NewBoolVar('shift_l%i s%i r%i' % (l, s, r))
    
    # Each lecture is assigned to exactly one room during a given shift
    for s in all_shifts:
        for r in all_rooms:
            model.Add(sum(shifts[(l, s, r)] for l in all_lectures) == 1)
    
    # Each lecture is allocated at most one room per shift
    for l in all_lectures:
         for s in all_shifts:
             model.Add(sum(shifts[(l, s, r)] for r in all_rooms) <= 1)

    min_rooms_per_lecture = (num_rooms * num_shifts) // num_lectures
    max_rooms_per_lecture = min_rooms_per_lecture + 1

    for l in all_lectures:
        num_lectures_allocated = sum(shifts[(l, s, r)] for s in all_shifts for r in all_rooms)
        model.Add(min_rooms_per_lecture <= num_lectures_allocated)
        model.Add(num_lectures_allocated <= max_rooms_per_lecture)
    
    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.parameters.linearization_level = 0
    
    # Display the first two solutions.
    a_few_solutions = range(2)
    solution_printer = RoomsPartialSolutionPrinter(
        shifts, num_rooms, num_lectures, num_shifts, a_few_solutions)
    solver.SearchForAllSolutions(model, solution_printer)
   
     # Statistics.
    print()
    print('Statistics')
    print('  - conflicts       : %i' % solver.NumConflicts())
    print('  - branches        : %i' % solver.NumBranches())
    print('  - wall time       : %f s' % solver.WallTime())
    print('  - solutions found : %i' % solution_printer.solution_count())


if __name__ == '__main__':
    main()

Solution 0
Shift 0
  Lecture 0 not allocated 
  Lecture 1 occupies room 1
  Lecture 2 occupies room 0
  Lecture 3 occupies room 2
  Lecture 4 occupies room 3
  Lecture 5 occupies room 4
Shift 1
  Lecture 0 occupies room 4
  Lecture 1 occupies room 3
  Lecture 2 not allocated 
  Lecture 3 occupies room 2
  Lecture 4 occupies room 1
  Lecture 5 occupies room 0
Shift 2
  Lecture 0 occupies room 3
  Lecture 1 occupies room 2
  Lecture 2 occupies room 1
  Lecture 3 occupies room 4
  Lecture 4 occupies room 0
  Lecture 5 not allocated 

Solution 1
Shift 0
  Lecture 0 not allocated 
  Lecture 1 occupies room 1
  Lecture 2 occupies room 0
  Lecture 3 occupies room 2
  Lecture 4 occupies room 3
  Lecture 5 occupies room 4
Shift 1
  Lecture 0 occupies room 4
  Lecture 1 not allocated 
  Lecture 2 occupies room 3
  Lecture 3 occupies room 2
  Lecture 4 occupies room 1
  Lecture 5 occupies room 0
Shift 2
  Lecture 0 occupies room 3
  Lecture 1 occupies room 2
  Lecture 2 occupies room 1
  Lecture 